In [ ]:
from pathlib import Path
import pandas as pd
from datetime import datetime

from openalea.mtg.io import write_mtg
from openalea.deploy.shared_data import shared_data
from oawidgets.plantgl import PlantGL

import openalea.strawberry
from openalea.strawberry.import_mtgfile import import_mtgfile,import_mtg_from_csv, union,strawberry_reader_csv
from openalea.strawberry import analysis
from openalea.strawberry import visu2d, visu3d
from openalea.plantgl.all import Viewer

In [ ]:
home_path = Path.home()
path_data= home_path / "Documents" / "GitHub" / "strawberry" / "share" / "data" / "breeding_value"
files=list(path_data.glob("*xlsx"))

In [4]:
shared_data(openalea.strawberry) / "breeding_value"

WindowsPath('C:/Users/Admin/miniconda3/envs/strawberry/Lib/site-packages/OpenAlea.Strawberry-1.2.0-py3.12.egg/share/data/breeding_value')

In [ ]:
mtgs=import_mtg_from_csv(files=files, first_property="experimental_names",symbol_at_scale=dict(P=1,T=2, F=3, f=3, b=3, HT=3, bt=3, ht=3,s=3))

In [ ]:
g=mtgs["Belle_et_Bonne"]
g= union(g)
#d=g.property('Sample_date')
from datetime import datetime
#datetime.strftime(d[1], '%d/%m/%Y')

for k in g.property('Sample_date').keys():
    g.property('Sample_date')[k]=datetime.strftime(g.property('Sample_date')[k], '%d/%m/%Y')
    g.property('Architecture_date')[k]=datetime.strftime(g.property('Architecture_date')[k], '%d/%m/%Y')
    

g.property_names()

In [ ]:
properties = [('experimental_name','STRING'),
              ('Genotype',"STRING"),
              ('Modality','STRING'),
              ('Plant_ID','REAL'),
              ('Sample_date','DD/MM/YY'),
              ('Architecture_date','DD/MM/YY'),
              ('Stade','STRING'),
              ('no_fruits','REAL'),
              ('no_flowers','REAL'),
              ('total_flowers','REAL'),
              ('area','REAL'),
              ('leaf_ID','REAL'),
              ('order','REAL')]
              
              
#properties = [(p, 'REAL') for p in g.property_names() if p not in ['edge_type', 'index', 'label','_axial_id']]

In [ ]:
mtg_lines = write_mtg(g, properties,class_at_scale={'P':1,'T':2,'A':2,'F':3,'f':3,'s':3,'bt':3,'ht':3,'HT':3},nb_tab=7)
filename = list(set(g.property("Genotype").values()))[0]+'.mtg'
data=path_data / filename
f = open(str(data), 'w',encoding='UTF8')
f.write(mtg_lines)
f.close()

In [ ]:
t= import_mtgfile(filename="Belle_et_Bonne")

In [ ]:
import_mtgfile?

# Main function

In [1]:
from pathlib import Path
import pandas as pd
from datetime import datetime

from openalea.mtg.io import write_mtg
from openalea.deploy.shared_data import shared_data
from oawidgets.plantgl import PlantGL

import openalea.strawberry
from openalea.strawberry.import_mtgfile import import_mtgfile,import_mtg_from_csv, union,strawberry_reader_csv
from openalea.strawberry import analysis
from openalea.strawberry import visu2d, visu3d
from openalea.plantgl.all import Viewer

In [2]:
def path_data(directory_name=None, filename=None):
    if directory_name is None:
        pathdir = shared_data(openalea.strawberry)
    else:
        pathdir= shared_data(openalea.strawberry) / directory_name
    
    if filename is None:
        pathdir = pathdir
    else:
        pathdir = str(pathdir)+'/'+str(filename)+'.xlsx'

    
    return pathdir

def write_excel_to_mtg(directory_name=None, 
                       filename=None, 
                       properties =[('experimental_name','STRING'),
                                    ('Genotype',"STRING"),
                                    ('Modality','STRING'),
                                    ('Plant_ID','REAL'),
                                    ('Sample_date','DD/MM/YY'),
                                    ('Architecture_date','DD/MM/YY'),
                                    ('Stade','STRING'),
                                    ('no_fruits','REAL'),
                                    ('no_flowers','REAL'),
                                    ('FLWRNUMBER','REAL'),
                                    ('area','REAL'),
                                    ('leaf_ID','REAL'),
                                    ('order','REAL')],
                       symbole_at_scale={"P":1,"T":2,"A":2,"F":3,"f":3,"HT":3, "bt":3, "ht":3,"s":3}):
    
    path_file=path_data(directory_name=directory_name, filename=filename)
    
    if str(path_file).endswith(".xlsx"):
        g=strawberry_reader_csv(file=path_file,first_property="experimental_name",symbole_at_scale=symbole_at_scale)
        g= union(g)
    else: 
        files=list(path_file.glob("*xlsx"))
        #assert (files== None, "directory no contain .xlsx file")
        g= import_mtg_from_csv(files=files,first_property="experimental_name",symbol_at_scale=symbole_at_scale)
        g=union(g)
        
    for k in g.property('Sample_date').keys():
        g.property('Sample_date')[k]=datetime.strftime(g.property('Sample_date')[k], '%d/%m/%Y')
        g.property('Architecture_date')[k]=datetime.strftime(g.property('Architecture_date')[k], '%d/%m/%Y')
    
    # convert total_flowers name in FLWRNUMBER (properties requiered in the source code)
    g.properties()["FLWRNUMBER"]=g.properties()["total_flowers"]
    del g.properties()["total_flowers"]
    
    max_order= max(g.property("order").values())+1
    mtg_lines = write_mtg(g, properties,class_at_scale=symbole_at_scale,nb_tab=max_order)
    name = path_file.replace('.xlsx','.mtg')
    
    f = open(str(name), 'w',encoding='UTF8')
    f.write(mtg_lines)
    f.close()

### For a single .xlsx file

In [ ]:
directory_name= "breeding_value" # name of your directory containing .xlsx file
filename= 'Belle_et_Bonne' # filename of .xlsx

write_excel_to_mtg(directory_name,filename,symbole_at_scale={'P':1,'T':2,'A':2,'F':3,'f':3,'s':3,'bt':3,'ht':3,'HT':3})

### For all file into one directory

In [3]:
directory_name= "breeding_value" #  name of your directory containing .xlsx file

path=shared_data(openalea.strawberry).glob(directory_name+'\*.xlsx')
names=[str(filename).split("\\")[-1].split(".")[0] for filename in path]
for name in names:
    write_excel_to_mtg(directory_name=directory_name,filename=name)

<>:3: SyntaxWarning: invalid escape sequence '\*'
<>:3: SyntaxWarning: invalid escape sequence '\*'
C:\Users\Admin\AppData\Local\Temp\ipykernel_15036\2080983354.py:3: SyntaxWarning: invalid escape sequence '\*'
  path=shared_data(openalea.strawberry).glob(directory_name+'\*.xlsx')
c:\Users\Admin\miniconda3\envs\strawberry\Lib\site-packages\OpenAlea.Strawberry-1.2.0-py3.12.egg\openalea\strawberry\import_mtgfile.py:223: Warning: Redeclaration of module 'F'.
  l.declare(module)
c:\Users\Admin\miniconda3\envs\strawberry\Lib\site-packages\OpenAlea.Strawberry-1.2.0-py3.12.egg\openalea\strawberry\import_mtgfile.py:223: Warning: Redeclaration of module 'f'.
  l.declare(module)
